# Machine Learning Project: French Grand Debat

### Project Introduction :

First, we would like to choose a problematic according to the data we collected. The aim of this project could be to answer the following questions:

* What are the 5 most important topics on each theme?
* Can we build different profiles of people with their ideas on the 4 themes?
 
#### Doability :

We are going to import the data, then check if the ids of people who submitted ideas on different themes are the same. If not, we won't be able to answer the second question.


In [11]:
# librairies imported
import src.utils as ut #read data
import numpy as np

#### Data extraction

In [4]:
df_fiscalite = ut.read_data('data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.json')
df_democratie = ut.read_data('data/DEMOCRATIE_ET_CITOYENNETE.json')
df_ecologie = ut.read_data('data/LA_TRANSITION_ECOLOGIQUE.json')
df_organisation = ut.read_data('data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json')

In [12]:
auth = df_fiscalite.loc[5, 'authorId']

dfs = np.array([["fiscalite", df_fiscalite], ["democratie", df_democratie], ["ecologie", df_ecologie], ["organisation", df_organisation]])
for df in dfs:
    print("in " + df[0] + " author " + auth + " :")
    print(df[1].loc[df[1]['authorId'] == auth, 'authorZipCode'])

in fiscalite author VXNlcjo3NmI3NTM2MS0xYjI2LTExZTktOTRkMi1mYTE2M2VlYjExZTE= :
5    66540
Name: authorZipCode, dtype: int64
in democratie author VXNlcjo3NmI3NTM2MS0xYjI2LTExZTktOTRkMi1mYTE2M2VlYjExZTE= :
19       66540
35225    66540
Name: authorZipCode, dtype: int64
in ecologie author VXNlcjo3NmI3NTM2MS0xYjI2LTExZTktOTRkMi1mYTE2M2VlYjExZTE= :
31    66540
Name: authorZipCode, dtype: int64
in organisation author VXNlcjo3NmI3NTM2MS0xYjI2LTExZTktOTRkMi1mYTE2M2VlYjExZTE= :
25       66540
66       66540
72       66540
38715    66540
38719    66540
Name: authorZipCode, dtype: int64


**From the previous lines, we can see that the `authorId` is likly to be an unique id whaterver the dataframe (zip code is the same).**

### How many people answerd the 4 themes ?

In [17]:
allAuthIds = set(dfs[0,1]['authorId'])
print(allAuthIds)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

